<h2><i>kiara</i>: Network Analysis</h2>

Welcome back! Now that we're comfortable with what <i>kiara</i> looks like and what it can do to help track your data and your research process, let's try out some of the digital analysis tools, starting with <b>Network Analysis</b>.

<h2>Why Network Analysis?</h2>

Network Analysis offers a computational and quantitative means to examine and explore relational objects, with proxies to measure structural roles and concepts such as power and influence. Doing so digitally - and at scale - also allows us to consider these kinds of questions with large amounts of material or documents that was not  heretofore manageable with qualitative or manual approaches.

>INSERT REFERENCES OR WIDER REFERENCING HERE: PROGRAMMING HISTORIAN LINKS W/ BIBLIOGRAPHY SECTION BELOW

<h3>Getting Started</h3>

Let's start by double checking that we have all the required plugins and setting up an API for us to use <i>kiara</i>. We'll do this all in one go this time, but if you're unsure, feel free to head back to the <a href="http://dharpa.org/kiara.documentation/latest/workshop/workshop/">installation notebook</a> to look over this section again.

In [18]:
import csv
import networkx as nx
import matplotlib.pyplot as plt

try:
    from kiara_plugin.jupyter import ensure_kiara_plugins
except:
    import sys
    print("Installing 'kiara_plugin.jupyter'...")
    !{sys.executable} -m pip install -q kiara_plugin.jupyter
    from kiara_plugin.jupyter import ensure_kiara_plugins

ensure_kiara_plugins()

from kiara.api import KiaraAPI
kiara = KiaraAPI.instance()

Great, we're all set up. We're going to import some data again like in the first notebook, but this time we're going to use a local file, using the kiara function `import.local.file`. We're using sample data again here, but you can also use this function to import your own data in the future. 

The data we're using here is a sample taken from the <b>correspSearch</b> dataset collated by the Berlin-Brandenburg Academy of Sciences and available on the 'LetterSampo' portal created by the Reassembling the Republic of Letters project team. For more on these projects see <a href="https://seco.cs.aalto.fi/projects/rrl/">here</a>.

A continually growing 'crowd-sourced' dataset, the correspSearch collection features letters from German history across a broad range of history, contributed by various individuals and larger research groups. Using quantitative network analysis on this dataset might offer insights into the most prolific writer in the dataset so-far, which actor connected the most people, or who operated in closely knit writing groups. Although we can also use network analysis to explore or 'map' our datasets if we don't know much about them, in this notebook the research questions and module parameters have been built around and defined by the information we already have about the data. It's important that we acknowledge this now as a core factor in the decisions already made for this process, but we'll also return to this throughout the notebook.

Let's use the <i>kiara</i> function `import.local.file` then to access our datset, specifying the path to the csv file in our <span style="color:green">inputs</span> and saving the <span style="color:red">outputs</span> of the function as '<b>correspsearch</b>'.

In [19]:
correspsearch = kiara.run_job('import.local.file', inputs={'path':'correspSearch.csv'})
correspsearch

╭──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                                                          │
│   field   value                                                                                                                          │
│  ─────────────────────────────────────────────────────────────────────────────────────                                                   │
│   file    Source,Target                                                                                                                  │
│           "Wagner, M.W.","Trew, Christoph Jacob"                                                                                         │
│           "Basellius, Nikolaus","Melanchthon, Philipp"                                                                                   │
│           "Marx, Karl F. H.",Jean Paul                                                                                                   │
│           Isacco Artom,Alfred Escher                                                                                                     │
│           Isacco Artom,Alfred Escher                                                                                                     │
│           Edoardo de Launay,Alfred Escher                                                                                                │
│           "Wedeke, Johann Christoph","Schleiermacher, Friedrich"                                                                         │
│           "Wedeke, Johann Christoph","Schleiermacher, Friedrich"                                                                         │
│           "Wedeke, Johann Christoph","Schleiermacher, Friedrich"                                                                         │
│           "Wedeke, Johann Christoph","Schleiermacher, Friedrich"                                                                         │
│           "Wedeke, Johann Christoph","Schleiermacher, Friedrich"                                                                         │
│           "Wedeke, Johann Christoph","Schleiermacher, Friedrich"                                                                         │
│           "Wedeke, Johann Christoph","Schleiermacher, Friedrich"                                                                         │
│           "Wedeke, Johann Christoph","Schleiermacher, Friedrich"                                                                         │
│           "Wedeke, Johann Christoph","Schleiermacher, Friedrich"                                                                         │
│           "Wedeke, Johann Christoph","Schleiermacher, Friedrich"                                                                         │
│           "Wedeke, Johann Christoph","Schleiermacher, Friedrich"                                                                         │
│           "Wedeke, Johann Christoph","Schleiermacher, Friedrich"                                                                         │
│           "Hoffmann, Johann Leonhard (1710-1782)","Haller, Albrecht von"                                                                 │
│           "Hoffmann, Johann Leonhard (1710-1782)","Haller, Albrecht von"                                                                 │
│           "Stupanus, Johann Niklaus","Camerarius, Joachim"                                                                               │
│           "Stupanus, Johann Niklaus","Camerarius, Joachim"                                                                               │
│           "Stupanus, Johann Niklaus","Herden, Balthasar von"                                                                             │
│           Jakob Benedikt Schmid,Alfred Escher     

<h2>Creating a Network</h2>

Time to make our network from this data. Let's start by searching for the <i>kiara</i> modules that are included in the `kiara_plugin.network_analysis` package.

In [20]:
infos = metadata = kiara.retrieve_operations_info()
operations = {}
for op_id, info in infos.item_infos.items():
    if info.context.labels.get("package", None) == "kiara_plugin.network_analysis":
        operations[op_id] = info

print(operations.keys())

dict_keys(['create.network_data.from.tables', 'export.network_data.as.csv_files', 'export.network_data.as.graphml_file', 'export.network_data.as.sql_dump', 'export.network_data.as.sqlite_db'])


There's lots of options for analysis, but we want to make our network first. Let's have a look what we need with the function `create.network_data.from.tables` using `kiara.retrieve_operation_info` once more.

<span style="color:blue">this will make more sense/have more options when analysis modules are incorporated into the network plugin (lol)</span>

In [21]:
kiara.retrieve_operation_info('create.network_data.from.tables')

Author(s)                                                                                                                                  
                     Markus Binsteiner   markus@frkl.io                                                                                     
                                                                                                                                            
 Context                                                                                                                                    
                     Tags         network_analysis                                                                                          
                     Labels       package: kiara_plugin.network_analysis                                                                    
                     References   source_repo: ]8;id=366974;https://github.com/DHARPA-Project/kiara_plugin.network_analysis\https://github.com/DHARPA-Project/kiara_plugin.network_analysis]8;;\                              
                                  documentation: ]8;id=79015;https://DHARPA-Project.github.io/kiara_plugin.network_analysis/\https://DHARPA-Project.github.io/kiara_plugin.network_analysis/]8;;\                            
                                                                                                                                            
 Operation details                                                                                                                          
                     Documentation   Create a graph object from one or two tables.                                                          
                                                                                                                                            
                                     This module needs at least one table as input, providing the edges of the resulting network data       
                                     set.                                                                                                   
                                     If no further table is created, basic node information will be automatically created by using unique   
                                     values from                                                                                            
                                     the edges source and target columns.                                                                   
                                                                                                                                            
                     Inputs                                                                                                                 
                                       field name           type     description                            Required   Default              
                                      ──────────────────────────────────────────────────────────────────────────────────────────────────    
                                       edges                table    A table that contains the edges        yes        -- no default --     
                                                                     data.                                                                  
                                       source_column_name   string   The name of the source column name     no         source               
                                                                     in the edges table.                                                    
                                       target_column_name   string   The name of the target column name     no         target               
                                                                     in the edges table.                                                    
                       

Like other network analysis tools, <i>kiara</i> first needs the data as an edge table. This means we first have to transform the csv file we imported earlier into a table before we can create the network data. Let's start by using the `create.table.from.file` function that we used in the first notebook and storing this as our <b>edges</b>, then use this to create our network data using the `create.network_data.from.tables` that we just read about. In this, we are defining two different sets of <span style="color:green">inputs</span>, overriding the first variable once we have used it to create our table.

If we want, we can also import a separate table with the nodes in, but this is optional, and for the moment let's stick with just the edge table. We'll store this again at the end in the variable <b>correspsearch</b> for us to use again in a bit.

<span style="color:blue">also include information on the f0/f1 problem, if this is not fixed by then - we'll also need to write in dropping the first row (i.e. 'Source' and 'Target') if this is the case. Otherwise, write instructions for identifying source/target columns, alongside more information on the rest of the possible inputs (so that it makes sense for the lesmis info later)</span>

In [22]:
inputs = {
    "file": correspsearch['file']
}

outputs = kiara.run_job('create.table.from.file', inputs={'file': correspsearch['file']})

edges = outputs['table']

inputs = {
    'edges': edges,
    'source_column_name': 'f0',
    'target_column_name': 'f1'
}

correspsearch = kiara.run_job('create.network_data.from.tables', inputs=inputs)
correspsearch

╭──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                                                          │
│   field          value                                                                                                                   │
│  ────────────────────────────────────────────────────────────────────────────────────────────────────────                                │
│   network_data                                                                                                                           │
│                  Table: edges                                                                                                            │
│                                                                                                                                          │
│                    source                                    target                                                                      │
│                   ───────────────────────────────────────────────────────────────────────                                                │
│                    Source                                    Target                                                                      │
│                    Wagner, M.W.                              Trew, Christoph Jacob                                                       │
│                    Basellius, Nikolaus                       Melanchthon, Philipp                                                        │
│                    Marx, Karl F. H.                          Jean Paul                                                                   │
│                    Isacco Artom                              Alfred Escher                                                               │
│                    Isacco Artom                              Alfred Escher                                                               │
│                    Edoardo de Launay                         Alfred Escher                                                               │
│                    Wedeke, Johann Christoph                  Schleiermacher, Friedrich                                                   │
│                    Wedeke, Johann Christoph                  Schleiermacher, Friedrich                                                   │
│                    Wedeke, Johann Christoph                  Schleiermacher, Friedrich                                                   │
│                    Wedeke, Johann Christoph                  Schleiermacher, Friedrich                                                   │
│                    Wedeke, Johann Christoph                  Schleiermacher, Friedrich                                                   │
│                    Wedeke, Johann Christoph                  Schleiermacher, Friedrich                                                   │
│                    Wedeke, Johann Christoph                  Schleiermacher, Friedrich                                                   │
│                    Wedeke, Johann Christoph                  Schleiermacher, Friedrich                                                   │
│                    Wedeke, Johann Christoph                  Schleiermacher, Friedrich                                                   │
│                    ...                                       ...                                                                         │
│                    ...                                       ...                                                                         │
│                    Peters                                    Spohr, Louis                                                                │
│                    Peters                         

Great - this has made a <i>kiara</i> network data object, and the output is showing the edge table and node table for the network. As we didn't give it a node table to start with this, it has extracted the information for the nodes from the edges instead.

As we can see, some of the edges are listed more than once, where more than one letter was written from one person to another. There's obviously more information about the network than just a list of edges and nodes then - let's find out some more information about our network object then.

<h2>Network Data</h2>

Let's start by having a look at the information for our network using the `get_property_data` function. As we're querying the `network_data` part of our network object, we'll save this as <b>correspsearch</b> for the moment.

In [23]:
correspsearch = correspsearch['network_data']
correspsearch.get_property_data('metadata.graph_properties')

 number_of_nodes            16523                                
  properties_by_graph_type   {                                    
                               "graph_type": "directed",          
                               "number_of_edges": 24249           
                             }                                    
                             {                                    
                               "graph_type": "undirected",        
                               "number_of_edges": 20325           
                             }                                    
                             {                                    
                               "graph_type": "directed-multi",    
                               "number_of_edges": 130121          
                             }                                    
                             {                                    
                               "graph_type": "undirected-multi",  
                               "number_of_edges": 130121          
                             }

Doing this gives us the total number of nodes, but also gives us an idea of the different kind of graphs we might chose to use for this dataset - <b>Directed</b>, <b>Undirected</b>, <b>Multi-Directed</b>, and <b>Multi-Undirected</b>. Having this kind of information accessible means we can make more informed decisions about the next steps that might work with our research or digital analysis, especially those that are sometimes automated for us.

Let's get some more information about the network as a whole then, using the `network_data.extract_largest_component` function. This works out how many different distinct components there are in a network, and also gives us the largest component on its own. We'll have a quick look at how it works first.

In [8]:
kiara.retrieve_operation_info('network_data.extract_largest_component')

Author(s)                                                                                                                                  
                     Lena Jaskov                helena.jaskov@uni.lu                                                                        
                     Mariella De Crouy Chanel   mariella.decrouychanel@uni.lu                                                               
                                                                                                                                            
 Context                                                                                                                                    
                     Tags         playground                                                                                                
                     Labels       package: kiara_plugin.playground                                                                          
                     References   source_repo: ]8;id=160807;https://github.com/DHARPA-Project/kiara_plugin.playground\https://github.com/DHARPA-Project/kiara_plugin.playground]8;;\                                    
                                  documentation: ]8;id=103090;https://DHARPA-Project.github.io/kiara_plugin.playground/\https://DHARPA-Project.github.io/kiara_plugin.playground/]8;;\                                  
                                                                                                                                            
 Operation details                                                                                                                          
                     Documentation   Extract the largest connected component from this network data.                                        
                                                                                                                                            
                                     This module analyses network data and checks if it contains clusters, and if so, how many. If all      
                                     nodes are connected                                                                                    
                                     to each other, the input data will be returned as largest component and the 'other_components'         
                                     output will be unset.                                                                                  
                                     Otherwise, the dataset will be split up into nodes of the largest component, and nodes that are not    
                                     part of that.                                                                                          
                                     Then this module will create 2 new network data items, one for the largest component, and one for      
                                     the other components that excludes                                                                     
                                     the nodes and edges that are part of the largest component.                                            
                                                                                                                                            
                     Inputs                                                                                                                 
                                       field name     type           description                            Required   Default              
                                      ──────────────────────────────────────────────────────────────────────────────────────────────────    
                                       network_data   network_data   The network data to analyze.           yes        -- no default --     
                                  

In [9]:
output = kiara.run_job('network_data.extract_largest_component', inputs={'network_data':correspsearch})
output

FailedJobException: Job failed.

'NetworkData' object has no attribute 'clone'

For now, let's save our largest component in the variable `network_data` for later - we'll use this for the rest of our experiments rather than the full network, and make sure we're tracing this using <i>kiara</i>.

Let's have a look at the information for this largest component, using our `get_property_data` function again.

In [13]:
#network_data = output['largest_component']

#network_data.get_property_data('metadata.graph_properties')

network_data = correspsearch

<h2>Onboarding Data: An Alternative</h2>

So far then, we have created a network object in <i>kiara</i> by importing a csv from a local path.

But what about other formats? Let's pause quickly, and have a look at importing a <b>gml</b> file instead. 

Here we will use a different sample dataset, <a href="http://www-personal.umich.edu/~mejn/netdata/">co-appearance network</a> of characters in Victor Hugo's novel <i>Les Miserables</i>, already in gml format. 

Let's have a look at the function `onboard.gml_file` and how this will work for us.

In [10]:
kiara.retrieve_operation_info('onboard.gml_file')

Author(s)                                                                                                                                  
                     Lena Jaskov                helena.jaskov@uni.lu                                                                        
                     Mariella De Crouy Chanel   mariella.decrouychanel@uni.lu                                                               
                                                                                                                                            
 Context                                                                                                                                    
                     Tags         playground                                                                                                
                     Labels       package: kiara_plugin.playground                                                                          
                     References   source_repo: ]8;id=212944;https://github.com/DHARPA-Project/kiara_plugin.playground\https://github.com/DHARPA-Project/kiara_plugin.playground]8;;\                                    
                                  documentation: ]8;id=378776;https://DHARPA-Project.github.io/kiara_plugin.playground/\https://DHARPA-Project.github.io/kiara_plugin.playground/]8;;\                                  
                                                                                                                                            
 Operation details                                                                                                                          
                     Documentation   This is a preliminary module for onboarding network data from gml files. It will likely be replaced    
                                     by more generic onboarding modules when those are ready.                                               
                                                                                                                                            
                                     Based on networkX deserialise GML file method:                                                         
                                     https://networkx.org/documentation/stable/reference/readwrite/generated/networkx.readwrite.gml.read…   
                                                                                                                                            
                     Inputs                                                                                                                 
                                       field name   type     description                                    Required   Default              
                                      ──────────────────────────────────────────────────────────────────────────────────────────────────    
                                       path         string   The path to the local file.                    yes        -- no default --     
                                       label        string   The node attribute that holds the 'label'      no         label                
                                                             information. Set this input to 'id' when                                       
                                                             there is no 'label' attribute in gml file.                                     
                                                                                                                                            
                                                                                                                                            
                     Outputs                                                                                                                
                                  

We need a local file path again, and we can go ahead and save this as <b>lesmis</b>.

In [11]:
lesmis = kiara.run_job('onboard.gml_file', inputs={'path':'lesmis.gml'})
lesmis

╭──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                                                          │
│   field          value                                                                                                                   │
│  ────────────────────────────────────────────────────────────                                                                            │
│   network_data                                                                                                                           │
│                  Table: edges                                                                                                            │
│                                                                                                                                          │
│                    source           target           value                                                                               │
│                   ─────────────────────────────────────────                                                                              │
│                    Myriel           Napoleon         1                                                                                   │
│                    Myriel           MlleBaptistine   8                                                                                   │
│                    Myriel           MmeMagloire      10                                                                                  │
│                    Myriel           CountessDeLo     1                                                                                   │
│                    Myriel           Geborand         1                                                                                   │
│                    Myriel           Champtercier     1                                                                                   │
│                    Myriel           Cravatte         1                                                                                   │
│                    Myriel           Count            2                                                                                   │
│                    Myriel           OldMan           1                                                                                   │
│                    Myriel           Valjean          5                                                                                   │
│                    MlleBaptistine   MmeMagloire      6                                                                                   │
│                    MlleBaptistine   Valjean          3                                                                                   │
│                    MmeMagloire      Valjean          3                                                                                   │
│                    Labarre          Valjean          1                                                                                   │
│                    Valjean          Marguerite       1                                                                                   │
│                    Valjean          MmeDeR           1                                                                                   │
│                    ...              ...              ...                                                                                 │
│                    ...              ...              ...                                                                                 │
│                    Bossuet          Grantaire        3                                                                                   │
│                    Bossuet          MmeHucheloup  

As we can see, this module not only imports the gml file into <i>kiara</i> but automatically converts it into a <i>kiara</i> network object for us. Great!

Here we can see that the edge table has a 'value' column to indicate edges weights that has also been automatically included with the gml data.

We'll leave this <i>Les Miserables</i> network for now, but it's useful to see this other option for importing data for networks. If you want to experiment with this dataset later, feel free to come back to it!

<h2>Network Analysis: Statistical Measures</h2>

Ok, let's head back to our correspSearch largest component dataset, stored in the variable <b>network_data</b>. We've already had a look at some graph wide measures, so let's start looking at some node specific measurements.

Let's start with degree, using `create.degree_rank_list`. This module allows us to calculate degree as both <b>undirected</b> and <b>weighted</b>. In this epistolary network, <b>undirected degree</b> counts the number of individual correspondents each person has, whereas <b>weighted degree</b> counts the total number of incoming and outgoing letters for each actor in the network. 

Let's use our `retrieve_operation_info` function to have a look at what we need to calculate these degrees.

In [12]:
kiara.retrieve_operation_info('create.degree_rank_list')

Author(s)                                                                                                                                  
                     Caitlin Burge              caitlin.burge@uni.lu                                                                        
                     Mariella De Crouy Chanel   mariella.decrouychanel@uni.lu                                                               
                                                                                                                                            
 Context                                                                                                                                    
                     Tags         playground                                                                                                
                     Labels       package: kiara_plugin.playground                                                                          
                     References   source_repo: ]8;id=276381;https://github.com/DHARPA-Project/kiara_plugin.playground\https://github.com/DHARPA-Project/kiara_plugin.playground]8;;\                                    
                                  documentation: ]8;id=947857;https://DHARPA-Project.github.io/kiara_plugin.playground/\https://DHARPA-Project.github.io/kiara_plugin.playground/]8;;\                                  
                                                                                                                                            
 Operation details                                                                                                                          
                     Documentation   Creates an ordered table with the rank and raw score for degree centrality.                            
                                                                                                                                            
                                     In an undirected graph, degree centrality measures the number of independent connections each node     
                                     has.                                                                                                   
                                                                                                                                            
                                     Uses networkx degree.                                                                                  
                                     https://networkx.org/documentation/stable/reference/generated/networkx.classes.function.degree.html    
                                                                                                                                            
                     Inputs                                                                                                                 
                                       field name           type           description                      Required   Default              
                                      ──────────────────────────────────────────────────────────────────────────────────────────────────    
                                       network_data         network_data   The network graph being          yes        -- no default --     
                                                                           queried.                                                         
                                       weighted_degree      boolean        Boolean to indicate whether to   no         True                 
                                                                           calculate weighted degree or                                     
                                                                           not. Conditions for                                              
                                  

So we've already computed the largest component to use as the `network_data` input, and we want to calculate the weighted degree meaures, so we'll leave the default as 'True'. Unlike the <i>Les Miserables</i> network, we don't have a pre-existing weight value for the edges, but we do know that there are parallell edges from multiple letters between correspondents, so we'll allow the module to aggregate the edges and set this as a weight. 

In creating this module, assumptions have already been made that we are working with both a single node type and a single edge type network. Again, a lot of the parameters have been set based on what we already know about the dataset, but we also need to acknowledge this as an active decision that has been 'pre-made' as part of the research process.

The inputs for `create.degree_rank_list` are prompting us to reflect on the decisions we are making as we are going along, and think about how our data fits into these kind of measurements, but by doing it in <i>kiara</i>, these inputs also allows us to <i>track</i> these decisions, as we will see more of later.

Let's give it a go then.

In [14]:
output = kiara.run_job('create.degree_rank_list', inputs={'network_data':network_data})
output

╭──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                                                          │
│   field                value                                                                                                             │
│  ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────  │
│   centrality_network                                                                                                                     │
│                        Table: edges                                                                                                      │
│                                                                                                                                          │
│                          source                                    target                                                                │
│                         ───────────────────────────────────────────────────────────────────────────────────                              │
│                          Gadendam, Johann Wilhelm                  Trew, Christoph Jacob                                                 │
│                          Münter, Friedrich                         Christoph Martin Wieland                                              │
│                          Münter, Friedrich                         Aloys Hirt                                                            │
│                          Johann Joachim Wolf                       Thomasius                                                             │
│                          Hainlin, Sebastian                        Heigel, Heinrich Magnus                                               │
│                          Hainlin, Sebastian                        Rhumel, Johann Conrad                                                 │
│                          Hainlin, Sebastian                        Queccius, Gregor                                                      │
│                          Hainlin, Sebastian                        Besler, Hieronymus                                                    │
│                          Hainlin, Sebastian                        Freher, Paul                                                          │
│                          Hainlin, Sebastian                        Volckamer, Johann Georg                                               │
│                          Hainlin, Sebastian                        Horn, Caspar                                                          │
│                          Hainlin, Sebastian                        Hofmann, Caspar                                                       │
│                          Hainlin, Sebastian                        Stella, Jeremias                                                      │
│                          Hainlin, Sebastian                        Weinlin, Josaphat                                                     │
│                          Hainlin, Sebastian                        Hilling, Gregor                                                       │
│                          Hainlin, Sebastian                        Leopold, Johann                                                       │
│                          ...                                       ...                                                                   │
│                          ...                                       ...                                                                   │
│                          Cratz, Anna Maria                         Lorang, Johann Peter                                                  │
│                          José Vidal               

In [15]:
kiara.retrieve_operation_info('create.betweenness_rank_list')

Author(s)                                                                                                                                  
                     Caitlin Burge              caitlin.burge@uni.lu                                                                        
                     Mariella De Crouy Chanel   mariella.decrouychanel@uni.lu                                                               
                                                                                                                                            
 Context                                                                                                                                    
                     Tags         playground                                                                                                
                     Labels       package: kiara_plugin.playground                                                                          
                     References   source_repo: ]8;id=932687;https://github.com/DHARPA-Project/kiara_plugin.playground\https://github.com/DHARPA-Project/kiara_plugin.playground]8;;\                                    
                                  documentation: ]8;id=903328;https://DHARPA-Project.github.io/kiara_plugin.playground/\https://DHARPA-Project.github.io/kiara_plugin.playground/]8;;\                                  
                                                                                                                                            
 Operation details                                                                                                                          
                     Documentation   Creates an ordered table with the rank and raw score for betweenness centrality.                       
                                                                                                                                            
                                     Betweenness centrality measures the percentage of all shortest paths that a node appears on,           
                                     therefore measuring the likeliness that a node may act as a connector or 'intermediary'.               
                                                                                                                                            
                                     Uses networkx.betweenness_centrality()                                                                 
                                     https://networkx.org/documentation/stable/reference/algorithms/generated/networkx.algorithms.centra…   
                                                                                                                                            
                     Inputs                                                                                                                 
                                       field name             type           description                    Required   Default              
                                      ──────────────────────────────────────────────────────────────────────────────────────────────────    
                                       network_data           network_data   The network graph being        yes        -- no default --     
                                                                             queried.                                                       
                                       weighted_betweenness   boolean        Boolean to indicate whether    no         False                
                                                                             to calculate weighted                                          
                                                                             betweenness as well as                                         
                                  

In [16]:
centrality_network = output['centrality_network']

output = kiara.run_job('create.betweenness_rank_list', inputs={'network_data':centrality_network, 'weight_column_name':'value', 'weighted_betweenness':True})

output

FailedJobException: Job failed.

(sqlite3.IntegrityError) NOT NULL constraint failed: nodes.Weighted Betweenness Score
[SQL: INSERT INTO nodes (label, "Weighted Degree Score", "Degree Score", "Weighted Betweenness Score", "Betweenness Score", id) VALUES (?, ?, ?, ?, ?, ?)]
[parameters: [('Eisenstein, Gotthold', 11, 1, 0.0, 0.0, 'Eisenstein, Gotthold'), ('Regierungsrath', 1, 1, 0.0, 0.0, 'Regierungsrath'), ('Buquoy, Georg von', 3, 1, 0.0, 0.0, 'Buquoy, Georg von'), ('Poullet-Delisle, Antoine Ch. M.', 1, 1, 0.0, 0.0, 'Poullet-Delisle, Antoine Ch. M.'), ('Stimmel, Johann Gottlob', 1, 1, 0.0, 0.0, 'Stimmel, Johann Gottlob'), ('Schneider, Gerhard', 10, 1, 0.0, 0.0, 'Schneider, Gerhard'), ('Leser der Allgemeinen Literaturzeitung', 1, 1, 0.0, 0.0, 'Leser der Allgemeinen Literaturzeitung'), ('Leutsch, Ernst von', 1, 1, 0.0, 0.0, 'Leutsch, Ernst von')  ... displaying 10 of 1024 total bound parameter sets ...  ('Endter, Wolfgang', 1, 1, 0.0, 0.0, 'Endter, Wolfgang'), ('Stegmann', 1, 1, 0.0, 0.0,

In [ ]:
kiara.retrieve_operation_info('create.eigenvector_rank_list')

In [ ]:
centrality_network = output['centrality_network']

output = kiara.run_job('create.eigenvector_rank_list', inputs={'network_data':centrality_network, 'weight_column_name':'value'})

output

In [ ]:
kiara.retrieve_operation_info('compute.modularity_group')

In [ ]:
network = output['centrality_network']

output = kiara.run_job('compute.modularity_group', inputs={'network_data':network})

output

In [ ]:
kiara.retrieve_operation_info('create.cut_point_list')

In [ ]:
centrality_network = output['centrality_network']

output = kiara.run_job('create.cut_point_list', inputs={'network_data':centrality_network})

output

In [ ]:
output['centrality_network'].lineage

In [ ]:
kiara.retrieve_operation_info('export.network_data.as.graphml_file')